In [69]:
import json
import os
import re
import yaml
from pathlib import Path
import pandas as pd

In [75]:
def load_names_data(names_file='names.yaml'):
    with open(names_file, 'r') as f:
        names_data = yaml.safe_load(f)

    name_lookup = {}

    for category, names in names_data.items():
        race, gender = category.split('_')

        for name in names:
            name_lookup[name] = (race.capitalize(), gender.capitalize())

    return name_lookup


def extract_negotiation_data(log_dir, name_lookup):
    """
    Extract buyer name, seller name, and final offer price from all game_state.json files
    in the given directory using the offer entries in game_state array.
    """
    results = []

    # Find all game_state.json files
    for subdir in sorted(os.listdir(log_dir)):
        subdir_path = os.path.join(log_dir, subdir)

        if not os.path.isdir(subdir_path):
            continue

        game_state_file = os.path.join(subdir_path, 'game_state.json')

        if not os.path.exists(game_state_file):
            continue

        try:
            with open(game_state_file, 'r') as f:
                data = json.load(f)

            # Extract settings from the first game state entry
            game_states = data.get('game_state', [])
            if not game_states:
                print(f"No game states found in {subdir}")
                continue

            settings = game_states[0].get('settings', {})
            buyer_name = settings.get('buyer_name', 'Unknown')
            seller_name = settings.get('seller_name', 'Unknown')
            buyer_first = settings.get('buyer_first')

            # Get race and gender from name_lookup
            seller_race, seller_gender = name_lookup.get(seller_name, ('Unknown', 'Unknown'))
            buyer_race, buyer_gender = name_lookup.get(buyer_name, ('Unknown', 'Unknown'))

            # Check if deal was accepted
            end_tag = data.get('end_tag', None)
            final_offer = None

            if end_tag == 'ACCEPT':
                # Find the last offer in the game_state entries
                # Iterate through game states in reverse to find the most recent offer
                for state in reversed(game_states[1:]):  # Skip the START state
                    public_info = state.get('player_public_info_dict', {})
                    offer = public_info.get('offer', '')

                    # Check if there's an offer value
                    if offer and offer.strip():
                        try:
                            final_offer = int(offer)
                            break
                        except ValueError:
                            continue

            results.append({
                'log_id': subdir,
                'seller_name': seller_name,
                'seller_race': seller_race,
                'seller_gender': seller_gender,
                'buyer_name': buyer_name,
                'buyer_race': buyer_race,
                'buyer_gender': buyer_gender,
                'buyer_first': buyer_first,
                'final_price': final_offer
            })

        except Exception as e:
            print(f"Error processing {game_state_file}: {e}")
            continue

    return results


In [76]:

name_lookup = load_names_data('names.yaml')

log_dir = '.logs/full_delegation'
results = extract_negotiation_data(log_dir, name_lookup)

res_df = pd.DataFrame.from_records(results)


In [77]:
res_df

,log_id,seller_name,seller_race,seller_gender,buyer_name,buyer_race,buyer_gender,buyer_first,final_price
0,1765432154136,Todd Becker,White,M,Dustin Becker,White,M,True,400000
1,1765432154151,Jake Becker,White,M,Hunter Becker,White,M,True,362000
2,1765432154163,Hunter Becker,White,M,Dustin Becker,White,M,True,362000
3,1765432154174,Hunter Becker,White,M,Matthew Becker,White,M,True,372000
4,1765432154184,Scott Becker,White,M,Zachary Becker,White,M,True,348000
...,...,...,...,...,...,...,...,...,...
115,1765432155421,Tremayne Washington,Black,M,DaQuan Washington,Black,M,True,382000
116,1765432155431,Terrell Washington,Black,M,Terrell Washington,Black,M,True,400000
117,1765432155440,Keyshawn Washington,Black,M,Keyshawn Washington,Black,M,True,366000
118,1765432155453,DeAndre Washington,Black,M,DaShawn Washington,Black,M,True,400000


In [78]:
res_df.query('seller_race == "Black"').final_price.describe()

count        60.000000
mean     369975.000000
std       24559.755043
min      300000.000000
25%      360750.000000
50%      368000.000000
75%      390000.000000
max      403000.000000
Name: final_price, dtype: float64

In [79]:
res_df.query('seller_race == "White"').final_price.describe()

count        60.000000
mean     368683.333333
std       19326.360032
min      300000.000000
25%      361000.000000
50%      363750.000000
75%      385000.000000
max      405000.000000
Name: final_price, dtype: float64